In [76]:
# DOC https://docs.djangoproject.com/en/4.2/ref/models/querysets/

from django.db import models
from paper.models import *

In [10]:
# fix notebook

import os, django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [12]:
from django.db import models
from paper.models import *

In [28]:
# выборка всех записей из таблицы paper
print(Paper.objects.all())
from django.db import connection
pprint(connection.queries[-1]['sql'])

<QuerySet [<Paper: покамир>, <Paper: приветмир>]>
('SELECT "paper_paper"."id", "paper_paper"."title", "paper_paper"."slug", '
 '"paper_paper"."content", "paper_paper"."photo", "paper_paper"."time_create", '
 '"paper_paper"."time_update", "paper_paper"."is_published", '
 '"paper_paper"."cat_id" FROM "paper_paper" ORDER BY '
 '"paper_paper"."time_create" DESC, "paper_paper"."title" ASC LIMIT 21')


In [29]:
# выборка срезом записей из таблицы Paper
print(Paper.objects.all()[0:1])
from django.db import connection
pprint(connection.queries[-1]['sql'])

<QuerySet [<Paper: покамир>]>
('SELECT "paper_paper"."id", "paper_paper"."title", "paper_paper"."slug", '
 '"paper_paper"."content", "paper_paper"."photo", "paper_paper"."time_create", '
 '"paper_paper"."time_update", "paper_paper"."is_published", '
 '"paper_paper"."cat_id" FROM "paper_paper" ORDER BY '
 '"paper_paper"."time_create" DESC, "paper_paper"."title" ASC LIMIT 1')


In [31]:
# Сортировка записей по первичному ключу 
Paper.objects.order_by('pk')

<QuerySet [<Paper: приветмир>, <Paper: покамир>]>

In [32]:
# обратный порядок сортировки записей по первичному ключу
Paper.objects.order_by('-pk')

<QuerySet [<Paper: покамир>, <Paper: приветмир>]>

In [41]:
# обратный порядок сортировки записей по первичному ключу (метод revese)
print(Paper.objects.all().reverse(), Paper.objects.order_by('-pk').reverse())

<QuerySet [<Paper: приветмир>, <Paper: покамир>]> <QuerySet [<Paper: приветмир>, <Paper: покамир>]>


In [44]:
# Выборка записей по ключу у которых ключ pk меньше либо равен 17
Paper.objects.filter(pk__lte=17)

<QuerySet [<Paper: приветмир>]>

In [50]:
# Выборка одной записи по ключу
item = Paper.objects.get(pk=18)
item

<Paper: покамир>

In [54]:
# item.row - значения записей таблицы Paper
# item.cat - объект класса Categories хранящий данные cat_id
item.cat

<Categories: вторая>

In [55]:
item.title

'покамир'

In [74]:
# Задача: Вывести все записи, которые относятся к категории "первая"

# Получаем объект категории
category = Categories.objects.get(pk=5)
# выводим список всех картегорий которые относятся к первой
category.paper_set.all()

# paper_set - можно поменять, если добавить в kwarg внутри модели Categories полю cat имя related_name='ur_name'

<QuerySet [<Paper: приветмир>]>

In [80]:
# lookups
# __gte - >=
# __lte - <=

Paper.objects.filter(pk__gte=17), Paper.objects.filter(pk__lte=17)

(<QuerySet [<Paper: покамир>, <Paper: приветмир>]>,
 <QuerySet [<Paper: приветмир>]>)

In [83]:
# Выбрать все записи, у которых в заголовке встречается фрагмент слова. Например "пока"
Paper.objects.filter(title__contains='пока')

<QuerySet [<Paper: покамир>]>

In [84]:
# Выбрать все записи как в примере выше, но без учета регистра
Paper.objects.filter(title__icontains='ЛИ')
# Результат будет пустым, потому что СУБД не поддерживает независимый поиск русских символов

<QuerySet []>

In [85]:
# Выбрать все записи у которых первичный ключ равен 17 и 18
Paper.objects.filter(pk__in=[17,18])

<QuerySet [<Paper: покамир>, <Paper: приветмир>]>

In [86]:
# Выбрать записи по примеру выше, только опубликованные
Paper.objects.filter(pk__in=[17,18], is_published=True)
# таким образом мы можем задавать несколько условий в фильтр

<QuerySet [<Paper: покамир>]>

In [88]:
# Выбрать все записи из таблицы которые равны N категориям
cats = Categories.objects.all()
Paper.objects.filter(cat_id__in=cats)

<QuerySet [<Paper: покамир>, <Paper: приветмир>]>

In [91]:
# Как формировать условия ( И ИЛИ НЕ ) ?
# Использовать класс Q
# & - 2 приоритет (и) 
# | - 3 приоритет (или)
# ~ - 1 приоритет (не)

from django.db.models import Q

In [94]:
# Задача: вывести сущности у которых pk равен 17 или title равен "покамир"
Paper.objects.filter(Q(pk=17) | Q(title='покамир'))

<QuerySet [<Paper: покамир>, <Paper: приветмир>]>

In [96]:
# Задача: вывести сущности у которых pk равен 17 и title равен "покамир"
Paper.objects.filter(Q(pk=17) & Q(title='покамир'))

<QuerySet []>

In [98]:
# Задача: вывести сущности у которых pk НЕ равен 17 и title равен "покамир"
Paper.objects.filter(~Q(pk=17) & Q(title='покамир'))

<QuerySet [<Paper: покамир>]>

In [101]:
# Задача: вывести первую сущность из таблицы
Paper.objects.first()

<Paper: покамир>

In [103]:
# Задача: вывести старую запись из таблицы
Paper.objects.latest('time_update')

<Paper: приветмир>

In [105]:
# Задача: вывести запись которая добавлена недавно в таблицу
Paper.objects.earliest('time_update')

<Paper: покамир>

In [106]:
# Сделать сортировку по заголовку и вывести запись, которая по времени была добавлена первой
Paper.objects.order_by('title').earliest('time_update')

<Paper: покамир>

In [115]:
# выбор записи относительно текущей (по дате)
x = Paper.objects.get(pk=17)
x.get_previous_by_time_update()

<Paper: покамир>

In [117]:
# exists() проверка существования записи
# count() получение числа записей

# Для тестов создаем новую категорию

Categories.objects.create(name="new_cat", slug="new_new")

<Categories: new_cat>

In [125]:
# выбираем данную рубрику
test_cat = Categories.objects.get(name='new_cat')
test_cat.paper_set.exists() # проверка существующих записей, которые причастны к данной категории

False

In [144]:
# Считаем количество записей
y = Categories.objects.all()
y
# x = y[..]
# x.paper_set.count()

<QuerySet [<Categories: new_cat>, <Categories: вторая>, <Categories: первая>]>


1

In [149]:
Paper.objects.filter(cat__name='первая')
Paper.objects.filter(cat__slug='vtoraya')

<QuerySet [<Paper: покамир>]>

In [156]:
# Выбрать все категории в которых есть статьи у которых в поле 'title' встречается 'ли'
Categories.objects.filter(paper__title__contains='п')

<QuerySet [<Categories: вторая>, <Categories: первая>]>

In [157]:
# взять только уникальные записи по примеру выше
Categories.objects.filter(paper__title__contains='п').distinct()

<QuerySet [<Categories: вторая>, <Categories: первая>]>

In [161]:
# Агрегирующие функции
# Задача: найти минимальный id
from django.db import *
Paper.objects.aggregate(Min('title'))
# также есть Max

{'title__min': 'покамир'}

In [164]:
# если есть желание поменять ключи, то нужно использовать следующий синтаксис
Paper.objects.aggregate(example=Min('title'))

{'example': 'покамир'}

In [165]:
# Выполнение математических операций
Paper.objects.aggregate(res=Sum('id') + Count('id'))

{'res': 37}

In [169]:
# выборка из объекта только определенных записей
Paper.objects.values('title').get(pk=17)

{'title': 'приветмир'}

In [175]:
# выбрать все категории и применить агрегирующие фукнции
Paper.objects.values('cat_id').annotate(total=Count('id'))

<QuerySet [{'cat_id': 5, 'total': 1}, {'cat_id': 6, 'total': 1}]>

In [179]:
# выбрать только те категории в которых записей больше 0
Paper.objects.values('cat_id').annotate(total=Count('id')).filter(total__gt=1)

<QuerySet [{'cat_id': 6, 'total': 2}]>

In [183]:
# Класс F
# Задача: сравнить первичный ключ с полем cat_id
from django.db.models import F
Paper.objects.filter(pk__gt=F('cat_id'))

<QuerySet [<Paper: onemore>, <Paper: покамир>, <Paper: приветмир>]>

In [184]:
Paper.objects.annotate()

<QuerySet [<Paper: onemore>, <Paper: покамир>, <Paper: приветмир>]>

In [187]:
from django.db import connections
connection.queries

[{'sql': 'SELECT "paper_paper"."id", "paper_paper"."title", "paper_paper"."slug", "paper_paper"."content", "paper_paper"."photo", "paper_paper"."time_create", "paper_paper"."time_update", "paper_paper"."is_published", "paper_paper"."cat_id" FROM "paper_paper" ORDER BY "paper_paper"."time_create" DESC, "paper_paper"."title" ASC LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "paper_paper"."id", "paper_paper"."title", "paper_paper"."slug", "paper_paper"."content", "paper_paper"."photo", "paper_paper"."time_create", "paper_paper"."time_update", "paper_paper"."is_published", "paper_paper"."cat_id" FROM "paper_paper" ORDER BY "paper_paper"."time_create" DESC, "paper_paper"."title" ASC LIMIT 21',
  'time': '0.000'},
 {'sql': 'SELECT "paper_paper"."id", "paper_paper"."title", "paper_paper"."slug", "paper_paper"."content", "paper_paper"."photo", "paper_paper"."time_create", "paper_paper"."time_update", "paper_paper"."is_published", "paper_paper"."cat_id" FROM "paper_paper" ORDER BY "paper_pape

In [194]:
# Вычисление длинны строки
from django.db.models.functions import Length
ps = Paper.objects.annotate(l=Length('title'))
ps[0].l

7

In [195]:
# RAW sql
# Manager.raw(<sql>)

In [200]:
w = Paper.objects.raw('SELECT * FROM paper_paper')
x = iter(w)
print(next(x))

приветмир


In [201]:
# очищаем последние SQL запросы
from django.db import reset_queries
reset_queries()
print(connection.queries)

[]


In [204]:
w = Paper.objects.raw('SELECT * FROM paper_paper')
w
print(connection.queries)

[]


In [205]:
# мы получили пустой список по причине
# до тех пор, пока мы не читаем переменную w ни одного SQL запроса сгенерировано не будет
# но если мы сейчас выполним следующее
w[0]

<Paper: приветмир>

In [207]:
# sql запрос будет сгенерирован
connection.queries

[{'sql': 'SELECT * FROM paper_paper', 'time': '0.000'}]

In [210]:
# когда мы прописываем конкретный sql запрос, то мы должны обязательно включать поле id иначе будет ошибка
w = Paper.objects.raw('SELECT id, title FROM paper_paper')

In [213]:
print(list(w))

[<Paper: приветмир>, <Paper: покамир>, <Paper: onemore>]


In [214]:
# учитывая, что мы делали запрос только на title. Мы можем также обратиться к другому полю

In [215]:
list(w)[0].is_published

True

In [216]:
# это произошло по причине того, что происходит "отложенная загрузка полей"
# происходит дополнительное обращение к базе данных для его получения

In [217]:
# хорошая практика для raw чтобы не было sql инъекций 
Paper.objects.raw("SELECT id, title from '%s'", [var])